In [ ]:
import requests
import ast
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from google.colab import files
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/player_metadata.csv')

In [ ]:
df

In [ ]:
Xf = df['expLevel']

In [ ]:
Ytemp = df['brawlers']
Ytemp

In [ ]:
Yresult = []
for i in range(len(Xf)):
    try:
        variable = Ytemp[i]

        start = variable.find("[")
        end = variable.rfind("]") + 1

        brawlers_data = variable[start:end]


        brawlers_list = ast.literal_eval(brawlers_data)

        power_11_count = sum(1 for brawler in brawlers_list if brawler.get('power') == 11)
        power_11_score = power_11_count / 65

        Yresult.append(power_11_score)

    except (SyntaxError, ValueError, Exception) as e:
        print(f"Error processing index {i}: {e}")
        Yresult.append(0)

print(Yresult)

In [ ]:
Yresults = pd.DataFrame(Yresult, columns = ['Power 11 Score'])
Yresults

In [ ]:
X = [Xf, Yresults]

In [ ]:
y = df['trophies']

In [ ]:
X = pd.concat([Xf, Yresults], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

In [ ]:
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f'Mean Absolute Percentage Error: {mape}%')

In [ ]:
new_data = pd.DataFrame([[198, 1]], columns=['expLevel', 'Power 11 Score'])
prediction = tree.predict(new_data)

print(f'Predicted number of trophies: {prediction[0]}')

In [ ]:
joblib.dump(tree, 'model.pkl')

In [ ]:
files.download('model.pkl')